In [3]:
f = open("SemEval2010_task8_training/TRAIN_FILE.TXT")

# read the train data line by line 
# store in a list
def readinFile(f):
    filelist = []
    line = f.readline()
    filelist.append(line)
    while line:
        line = f.readline()
        filelist.append(line)
    return filelist

filelist = readinFile(f)

In [4]:
# using 3 lists to store 3 elements
# list_sentence: each sentence in the list
# list_class: each class in the list
# list comment: each comment in the list
def separateFilelist(l):
    list_sentence = []
    list_class = []
    list_comment = []
    for i in range(len(l)):
        if (i + 1) % 4 == 1:
            list_sentence.append(l[i])
        if (i + 1) % 4 == 2:
            list_class.append(l[i])
        if (i + 1) % 4 == 3:
            list_comment.append(l[i])
    # to remove the '' in the end, no use
    list_sentence.remove('')
    return list_sentence,list_class,list_comment

list_sentence,list_class,list_comment = separateFilelist(filelist)

# here we delete the sentences which give us noises
# index: 212,2739,4218,4611,4783,6372

list_sentence.remove(list_sentence[212])
list_sentence.remove(list_sentence[2738])
list_sentence.remove(list_sentence[4216])
list_sentence.remove(list_sentence[4608])
list_sentence.remove(list_sentence[4779])
list_sentence.remove(list_sentence[6367])

list_class.remove(list_class[212])
list_class.remove(list_class[2738])
list_class.remove(list_class[4216])
list_class.remove(list_class[4608])
list_class.remove(list_class[4779])
list_class.remove(list_class[6367])

list_comment.remove(list_comment[212])
list_comment.remove(list_comment[2738])
list_comment.remove(list_comment[4216])
list_comment.remove(list_comment[4608])
list_comment.remove(list_comment[4779])
list_comment.remove(list_comment[6367])

In [6]:
def sizeoftrain(list_sentence,list_class,list_comment):
    print("The length of the sentences is: ",len(list_sentence))
    print("The length of the classes is: ",len(list_class))
    print("The length of the comments is: ",len(list_comment))
    
sizeoftrain(list_sentence,list_class,list_comment)

The length of the sentences is:  7994
The length of the classes is:  7994
The length of the comments is:  7994


In [7]:
from bs4 import BeautifulSoup
def getNouns(list_sentence):
    l_noun = []
    for sentence in list_sentence:
        flag = 0
        l1 = ''
        l2 = ''
        couple_noun = []
        for i in range(len(sentence)):
            if (sentence[i] == '<') and (sentence[i+1] == 'e') and (sentence[i+2] == '1'):
                flag = 1
            if (sentence[i] == '>') and (sentence[i-1] == '1') and (sentence[i-2] == 'e') and (sentence[i-3] == '/'):
                flag = 0
            if (sentence[i] == '<') and (sentence[i+1] == 'e') and (sentence[i+2] == '2'):
                flag = 2
            if (sentence[i] == '>') and (sentence[i-1] == '2') and (sentence[i-2] == 'e') and (sentence[i-3] == '/'):
                flag = 0

            if flag == 1:
                l1 += sentence[i]
            if flag == 2:
                l2 += sentence[i]
        noun1_b = BeautifulSoup(l1)
        noun2_b = BeautifulSoup(l2)
        noun1 = noun1_b.get_text()
        noun2 = noun2_b.get_text()
        couple_noun.append(noun1)
        couple_noun.append(noun2)
        l_noun.append(couple_noun)
    return l_noun

l_noun = getNouns(list_sentence)

/home/sirius/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /home/sirius/anaconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [8]:
# replace the space in the marked nous into #, and replace these nous in the sentences
# example: WANG Yaohui -> WANG#Yaohui

import nltk.data
import re

def replace_the_space(l_noun):
    l_new_noun = []
    for couple in l_noun:
        c = (couple[0].replace(' ','#'),couple[1].replace(' ','#'))
        l_new_noun.append(c)
    return l_new_noun

def replace_original_noun(list_sentence,l_new_noun):
    l_new_sentence = []
    for sentence,couple in zip(list_sentence,l_new_noun):
        left = couple[0].replace('#',' ')
        right = couple[1].replace('#',' ')
        sentence = re.sub("<e1>" + left + "</e1>",couple[0],sentence)
        sentence = re.sub("<e2>" + right + "</e2>",couple[1],sentence)
        l_new_sentence.append(sentence)
    return l_new_sentence

def get_the_new_sentence(list_sentence,l_noun):
    final_sentence = []
    
    l_new_noun = replace_the_space(l_noun)
    l_new_sentence = replace_original_noun(list_sentence,l_new_noun)
    
    for l in l_new_sentence:
        sentence_text = re.sub("[^a-zA-Z#-]"," ",l)
        words = sentence_text.lower().split()
        final_sentence.append(words)
    return final_sentence

def get_the_final_couple(l_noun):
    new_couple = replace_the_space(l_noun)
    final_couple = []
    for couple in new_couple:
        c = (couple[0].lower(),couple[1].lower())
        final_couple.append(c)
    return final_couple

# final sentences we need
final_sentence = get_the_new_sentence(list_sentence,l_noun)
# final marked nouns we need
final_couple = get_the_final_couple(l_noun)

In [9]:
def get_location(final_sentence,final_couple):
    list_index = []
    for couple,sentence in zip(final_couple,final_sentence):
        #print(sentence)
        #print(couple[0] + couple[1])
        noun_couple = (sentence.index(couple[0]),sentence.index(couple[1]))
        list_index.append(noun_couple)
    return list_index

# location of marked nouns in the sentences
list_index = get_location(final_sentence, final_couple)

In [10]:
# apply word2vec on our final sentences list
from gensim import models
import logging
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s', level = logging.INFO)

num_features = 50
min_word_count = 1
num_workers = 8
context = 10
downsampling = 0.0001
print("Training model...")

model = models.word2vec.Word2Vec(final_sentence, workers = num_workers, size = num_features, min_count = min_word_count, window = context, sample = downsampling)

2017-01-24 22:01:26,887: WARNING: consider setting layer size to a multiple of 4 for greater performance
2017-01-24 22:01:26,887: INFO: collecting all words and their counts
2017-01-24 22:01:26,888: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-01-24 22:01:26,930: INFO: collected 19818 word types from a corpus of 136793 raw words and 7994 sentences
2017-01-24 22:01:26,931: INFO: Loading a fresh vocabulary
2017-01-24 22:01:27,026: INFO: min_count=1 retains 19818 unique words (100% of original 19818, drops 0)
2017-01-24 22:01:27,026: INFO: min_count=1 leaves 136793 word corpus (100% of original 136793, drops 0)
2017-01-24 22:01:27,082: INFO: deleting the raw counts dictionary of 19818 items
2017-01-24 22:01:27,083: INFO: sample=0.0001 downsamples 330 most-common words
2017-01-24 22:01:27,084: INFO: downsampling leaves estimated 79036 word corpus (57.8% of prior 136793)
2017-01-24 22:01:27,084: INFO: estimated required memory for 19818 words and 50 dimension

Training model...


2017-01-24 22:01:27,140: INFO: resetting layer weights
2017-01-24 22:01:27,358: INFO: training model with 8 workers on 19818 vocabulary and 50 features, using sg=0 hs=0 sample=0.0001 negative=5 window=10
2017-01-24 22:01:27,359: INFO: expecting 7994 sentences, matching count from corpus used for vocabulary survey
2017-01-24 22:01:27,764: INFO: worker thread finished; awaiting finish of 7 more threads
2017-01-24 22:01:27,773: INFO: worker thread finished; awaiting finish of 6 more threads
2017-01-24 22:01:27,774: INFO: worker thread finished; awaiting finish of 5 more threads
2017-01-24 22:01:27,790: INFO: worker thread finished; awaiting finish of 4 more threads
2017-01-24 22:01:27,791: INFO: worker thread finished; awaiting finish of 3 more threads
2017-01-24 22:01:27,795: INFO: worker thread finished; awaiting finish of 2 more threads
2017-01-24 22:01:27,796: INFO: worker thread finished; awaiting finish of 1 more threads
2017-01-24 22:01:27,800: INFO: worker thread finished; awaitin

In [11]:
model.init_sims(replace = True)

model_name = "wia"
model.save(model_name)

2017-01-24 22:01:37,849: INFO: precomputing L2-norms of word weight vectors
2017-01-24 22:01:37,966: INFO: saving Word2Vec object under wia, separately None
2017-01-24 22:01:37,967: INFO: not storing attribute syn0norm
2017-01-24 22:01:37,968: INFO: not storing attribute cum_table
2017-01-24 22:01:38,074: INFO: saved wia


In [12]:
model.doesnt_match("man waman child kitchen".split())

'child'

In [13]:
import numpy as np

# L1 L2 L3 L4
def get_lexical_feature(final_couple,list_sentence,final_sentence):
    lexical_feature = []
    for couple,index,sentence in zip(final_couple,list_index,final_sentence):

        noun1_v = model[couple[0]]
        noun2_v = model[couple[1]]
    
        if index[0] == 0:
            noun3_v = model[sentence[index[0]+1]] / 2 
        if index[0] == (len(sentence)-1):
            noun3_v = model[sentence[index[0]-1]] / 2
        else:
            noun3_v = (model[sentence[index[0]-1]] + model[sentence[index[0]+1]]) / 2
        
        if index[1] == 0:
            noun4_v = model[sentence[index[1]+1]] / 2 
        if index[1] == (len(sentence)-1):
            noun4_v = model[sentence[index[1]-1]] / 2
        else:
            noun4_v = (model[sentence[index[1]-1]] + model[sentence[index[1]+1]]) / 2
    
        word_feature = np.concatenate((noun1_v,noun2_v,noun3_v,noun4_v))
        lexical_feature.append(word_feature)
    return np.array(lexical_feature)

lexical_feature = get_lexical_feature(final_couple,list_sentence,final_sentence)

In [14]:
lexical_feature.shape

(7994, 200)

## sentence features

In [15]:
# padding make all the sentences be the same length, using </s> to padding 
def pad_sentences(sentences, padding_word="</s>"):
    padding_word = "</s>"
    sequence_length = max(len(x) for x in sentences) # get the max length in the dataset
    padded_sentences = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        num_padding = sequence_length - len(sentence)
        new_sentence = sentence + [padding_word] * num_padding
        padded_sentences.append(new_sentence)
    return padded_sentences

sentences_padded = pad_sentences(final_sentence)

In [18]:
import numpy as np
# define the beginning and the end 
xs = np.zeros(50)
xe = np.zeros(50)
pad = np.zeros(50)

In [25]:
# get the [WF, PF]
def get_the_sentence_feature(sentences_padded,list_index):
    window = 3
    list_sentence_feature = []
    for sentence,position in zip(sentences_padded,list_index):
        wf_l = []
        for index in range(len(sentence)):
            if index == 0:
                if sentence[index] == '</s>':
                    a = pad
                else:
                    a = model[sentence[index]]
                if sentence[index+1] == '</s>':
                    b = pad
                else:
                    b = model[sentence[index+1]]
                wf = np.concatenate((xs, a, b, [index-position[0]], [index-position[1]]))
            if index == (len(sentence)-1):
                if sentence[index-1] == '</s>':
                    a = pad
                else:
                    a = model[sentence[index-1]]
                if sentence[index] == '</s>':
                    b = pad
                else:
                    b = model[sentence[index]]
                wf = np.concatenate((a, b, xe, [index-position[0]], [index-position[1]]))
            else:
                if sentence[index-1] == '</s>':
                    a = pad
                else:
                    a = model[sentence[index-1]]
                if sentence[index] == '</s>':
                    b = pad
                else:
                    b = model[sentence[index]]
                if sentence[index+1] == '</s>':
                    c = pad
                else:
                    c = model[sentence[index+1]]
                wf = np.concatenate((a, b, c, [index-position[0]], [index-position[1]]))
            wf_l.append(wf)
        list_sentence_feature.append(np.array(wf_l))
    return list_sentence_feature

list_sentence_feature = get_the_sentence_feature(sentences_padded,list_index)

In [85]:
X = np.array(list_sentence_feature)

In [86]:
X = X.reshape(X.shape[0],1,86,152)

In [87]:
X.shape

(7994, 1, 86, 152)

In [28]:
from keras.utils import np_utils
import pandas as pd

y_class = pd.get_dummies(np.array(list_class)).values.argmax(1)
y = np_utils.to_categorical(y_class, 19)

Using Theano backend.
WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: unknown error)
2017-01-24 22:16:18,188: WARNING: CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: unknown error)


In [29]:
y.shape

(7994, 19)

## train model DCNN

In [91]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling2D,MaxPooling1D
from keras.utils import np_utils
from keras import backend as K
from keras.layers import Input,merge
from keras.models import Model

In [92]:
main_input = Input(shape=(1,86,152))
# conv1 = Convolution1D(32,3,activation='relu')(main_input)
hidden1 = Dense(200,activation='linear')(main_input)
pool = MaxPooling2D(pool_size = (1,200),dim_ordering='th')(hidden1)
flatten = Flatten()(pool)
hidden = Dense(100,activation='tanh')(flatten)
# add lexical features
lexical_input = Input(shape=(200,), name='lexical_input')
x = merge([lexical_input, hidden], mode='concat')
out = Dense(19,activation='softmax')(x)

In [93]:
model = Model(input=[main_input,lexical_input],output=out)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss = 'categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit([X,lexical_feature], y, batch_size=20, nb_epoch=10,verbose=1)

Epoch 1/10
7994/7994 [==============================] - 3s - loss: 2.7094 - acc: 0.1709     
Epoch 2/10
7994/7994 [==============================] - 4s - loss: 2.7021 - acc: 0.1715     
Epoch 3/10
7994/7994 [==============================] - 4s - loss: 2.6943 - acc: 0.1759     
Epoch 4/10
7994/7994 [==============================] - 4s - loss: 2.6920 - acc: 0.1736     
Epoch 5/10
7994/7994 [==============================] - 4s - loss: 2.6902 - acc: 0.1748     
Epoch 6/10
7994/7994 [==============================] - 4s - loss: 2.6944 - acc: 0.1758     
Epoch 7/10
7994/7994 [==============================] - 4s - loss: 2.6933 - acc: 0.1726     
Epoch 8/10
7994/7994 [==============================] - 5s - loss: 2.6860 - acc: 0.1729     
Epoch 9/10
7994/7994 [==============================] - 5s - loss: 2.6852 - acc: 0.1754     
Epoch 10/10
7994/7994 [==============================] - 5s - loss: 2.6856 - acc: 0.1746     
